# Data Ingestion

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# Txt Reader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("example.txt")
text_docs = loader.load()
text_docs

[Document(metadata={'source': 'example.txt'}, page_content='Hola mi nombre es Pedro, soy un Picapiedra.\n\nMe gusta comer hamburguesa y jugar al fútbol.')]

In [10]:
# PDF Reader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("example_2.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'type': '/Metadata', 'subtype': '/XML', 'source': 'example_2.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Usuarios\n*UsuarioID\nUsuarioNombre\nUsuarioEmail\nUsuarioRol\nUsuarioStatus\nProductos\n*ProductoID\nProductoNombre\nProductoPrecio\nProductoStockMinimo\nProductoFoto\nUnidadMedidaID\nProductoStatus\nClientes\n*ClienteID\nClienteNombre\nClienteDireccion\nClienteTelefono\nClienteStatus\nProveedores\n*ProveedorID\nProveedorNombre\nProveedorCategoria\nProveedorTelefono\nProveedorStatus\nUnidadesMedida\n*UnidadMedidaID\nUnidadMedidaNombre\nUnidadMedidaStatus\nVentas\n*VentaID\nVentaFolio\nVentaFecha\nVentaHora\nClienteID\nUsuarioID\nVentaMetodoPago\nVentaStatus\nVentaDetalle\n*VentaDetalleID\nVentaID\nProductoID\nVentaDetalleCantidad\nVentaDetallePrecio\nVentaDetalleImporte\nEntradaMaterial\n \n*EntradaMaterialID\nEntradaMaterialFolio\nEntradaMaterialFecha\nEntradaMaterialHora\nUsu

In [ ]:
# Web Reader con Beautiful Soup para WebScrapping
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

# Transform

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=5)
documents=text_splitter.split_documents(text_docs)
documents

[Document(metadata={'source': 'example.txt'}, page_content='Hola mi nombre es Pedro, soy un Picapiedra.'),
 Document(metadata={'source': 'example.txt'}, page_content='Me gusta comer hamburguesa y jugar al fútbol.')]

In [37]:
# Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings(model="text-embedding-3-small"))

In [38]:
query = "¿Qué le gusta comer a Pedro?"
retireved_results = db.similarity_search(query)
print(retireved_results[0].page_content)

Me gusta comer hamburguesa y jugar al fútbol.


In [40]:
# OpenAI ChatGPT LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [45]:
# Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Responde la siguiente preguna basado solamente en el contexto dado.
Piensa paso a paso antes de entregar una respuesta detallada.
<context>
{context}
</context>
Question: {input}""")

In [ ]:
# Chain Introduction
# Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

In [42]:
from langchain.chains import create_retrieval_chain
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [46]:
response = retrieval_chain.invoke({"input":"Me gustaría saber que comida le gusta a Pedro"})

In [47]:
response["answer"]

'Pedro menciona que le gusta comer hamburguesa en el texto proporcionado. Por lo tanto, la comida que le gusta a Pedro es la hamburguesa.'